# Big Data Modeling and Management Assigment Neo4j

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 5-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "/Users/CarlotaCarneiro/Desktop/NOVA IMS/BDMM/Práticas/Aula 1/Neo4Jplugins":/plugins -v "/Users/CarlotaCarneiro/Desktop/NOVA IMS/BDMM/HW/Neo4JHWData/data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted="gds.*" --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**3**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|20220658|Alex Santander|
|20210684|Carlota Carneiro Vieira|
|20220052|Inês Rocha|
|20210995|Ricardo Arraiano|
|20221041|Skander Chaabini|

#### 1. How many different countries exist in the database?

In [7]:
query_1 = """
            MATCH (c:COUNTRIES)
            RETURN COUNT(c) as Number_of_Countries
          """

result_1 = execute_read(driver, query_1)

pprint(result_1)

[<Record Number_of_Countries=200>]


#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [8]:
query_2a = """
            MATCH (b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
            RETURN b.name as Beer, COUNT(r) as Number_of_Reviews
            ORDER BY Number_of_Reviews DESC
            LIMIT 1
           """

result_2a = execute_read(driver, query_2a)

pprint(result_2a)

[<Record Beer='IPA' Number_of_Reviews=31387>]


In [9]:
query_2b = """
            MATCH (br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
            RETURN br.name as Brewery, COUNT(r) as Number_of_Reviews
            ORDER BY Number_of_Reviews DESC
            LIMIT 1
           """

result_2b = execute_read(driver, query_2b)

pprint(result_2b)

[<Record Brewery='Sierra Nevada Brewing Co.' Number_of_Reviews=175161>]


In [10]:
query_2c = """
            MATCH (c:COUNTRIES)<-[in:IN]-(ci:CITIES)<-[ins:IN]-(br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
            RETURN c.name as Country, COUNT(r) as Number_of_Reviews
            ORDER BY Number_of_Reviews DESC
            LIMIT 1
           """

result_2c = execute_read(driver, query_2c)

pprint(result_2c)

[<Record Country='US' Number_of_Reviews=7675804>]


#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [11]:
name = 'CTJman'

query_3 = f"""
            MATCH (u1:USER{{name: '{name}'}})<-[p1:POSTED]-(r1:REVIEWS)<-[rv1:REVIEWED]-(b:BEERS)-[rv2:REVIEWED]->(r2:REVIEWS)-[p2:POSTED]->(u2:USER)
            RETURN u2.name as Username, count(u2) as Shared_Reviews
            ORDER BY Shared_Reviews DESC
            LIMIT 1
           """

result_3 = execute_read(driver, query_3)

pprint(result_3)

[<Record Username='acurtis' Shared_Reviews=1428>]


In [12]:
# Checking the total number of reviews of the User 'acurtis'

name = result_3[0][0]

query_3a = f"""
             MATCH (u1:USER{{name: '{name}'}})<-[p1:POSTED]-(r1:REVIEWS)<-[rv1:REVIEWED]-(b:BEERS)
             RETURN u1.name as Username, COUNT(r1) as Number_of_Reviews
            """

result_3a = execute_read(driver, query_3a)

pprint(result_3a)

[<Record Username='acurtis' Number_of_Reviews=12016>]


#### 4. Which Portuguese brewery has the most beers?

In [13]:
query_4 = """
            MATCH (c:COUNTRIES)<-[in:IN]-(ci:CITIES)<-[ins:IN]-(br:BREWERIES)-[bre:BREWED]->(b:BEERS)
            WHERE c.name = "PT"
            RETURN br.name as Brewery, COUNT(b) as Number_of_Beers
            ORDER BY Number_of_Beers DESC
            LIMIT 1
          """

result_4 = execute_read(driver, query_4)

pprint(result_4)

[<Record Brewery='Dois Corvos Cervejeira' Number_of_Beers=40>]


#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [14]:
query_5 = f"""
            MATCH (br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
            WHERE br.name = '{result_4[0][0]}'
            RETURN b.name as Beer, COUNT(r) as Number_of_Reviews
            ORDER BY Number_of_Reviews DESC
            LIMIT 1
           """

result_5 = execute_read(driver, query_5)

pprint(result_5)

[<Record Beer='Finisterra' Number_of_Reviews=10>]


#### 6. On average how many different beer styles does each brewery produce?

In [15]:
query_6 = """
            MATCH (br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[h:HAS_STYLE]->(s:STYLE)
            WITH br, COUNT(distinct s.name) as Beer_Styles
            RETURN ROUND(AVG(Beer_Styles), 2) as Average_Number_of_Beer_Styles
          """

result_6 = execute_read(driver, query_6)

pprint(result_6)

[<Record Average_Number_of_Beer_Styles=10.6>]


#### 7. Which brewery produces the strongest beers according to ABV?

<b> Answer:</b>

In [16]:
# Brewery with the strongest beers according to ABV (on average)

query_7a = """
            MATCH (br:BREWERIES)-[bre:BREWED]->(b:BEERS)
            WHERE toFloat(b.abv) <> 'nan'
            RETURN DISTINCT br.name as Brewery, ROUND(AVG(toFloat(b.abv)), 2) as Average_ABV
            ORDER BY Average_ABV DESC
            LIMIT 1
           """

result_7a = execute_read(driver, query_7a)

pprint(result_7a)

[<Record Brewery='1648 Brewing Company Ltd' Average_ABV=25.58>]


<b> Extra Answer: </b>

In [17]:
# Brewery with the strongest beer according to ABV (maximum)

query_7b = """
            MATCH (be:BEERS)
            WHERE toFloat(be.abv) <> 'nan'
            WITH MAX(toFloat(be.abv)) as maximum_abv
            MATCH (br:BREWERIES)-[bre:BREWED]->(b:BEERS)
            WHERE toFloat(b.abv) = maximum_abv
            RETURN DISTINCT br.name as Brewery, toFloat(b.abv) as ABV
           """

result_7b = execute_read(driver, query_7b)

pprint(result_7b)

[<Record Brewery='Avondale Brewing Co.' ABV=100.0>,
 <Record Brewery='1648 Brewing Company Ltd' ABV=100.0>]


#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [18]:
# We grouped the beers by their styles and then we averaged the smell score for the aroma and the look score for the appearance.
# We ordered the styles based on the sum of both requisites and chose the top 1
query_8 = """
            MATCH (s: STYLE)<-[h:HAS_STYLE]-(b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
            WHERE toFloat(r.smell) <> 'nan' AND toFloat(r.look) <> 'nan'
            RETURN DISTINCT s.name as Style, ROUND(AVG(toFloat(r.smell)), 2) as Average_Aroma, ROUND(AVG(toFloat(r.look)), 2) as Average_Look
            ORDER BY (Average_Aroma + Average_Look) DESC
            LIMIT 1
          """

result_8 = execute_read(driver, query_8)

pprint(result_8)

[<Record Style='New England IPA' Average_Aroma=4.41 Average_Look=4.38>]


#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?  
    --> Node Similarity algorithm
    2. Which beer is the most influential when considering beers are conected by users who review them?  
    --> PageRank
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?  
    --> Label Propagation algorithm

<b>9.1.</b> Which two countries are most similiar when it comes to their top 10 most produced Beer styles?

In [19]:
# Step 0 - Clear graph

try:
    query_9a = """
            CALL gds.graph.drop('similar_countries') YIELD graphName;
        """

    result_9a = execute_read(driver, query_9a)

    pprint(result_9a)
    
except Exception as e:
    pprint(e)

[<Record graphName='similar_countries'>]


In [20]:
# Step 1 - Create the similar_countries graph

try:
    query_9b = """
         CALL gds.graph.project.cypher(
            'similar_countries',
            'MATCH (n:COUNTRIES) RETURN id(n) AS id UNION ALL MATCH (s:STYLE) return id(s) as id',
            'MATCH (c:COUNTRIES)<-[]-(ci:CITIES)<-[]-(br:BREWERIES)-[]->(be:BEERS)-[]->(s:STYLE) WITH c as Country, s as Style, count(*) as Freq
                ORDER BY Freq DESC
                WITH Country, collect(Style)[..10] as Top_10 UNWIND Top_10 as Country_Top_10 
                RETURN id(Country) as source, id(Country_Top_10) as target'
        )
        """

    result_9b = execute_read(driver, query_9b)

    pprint(result_9b)
    
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (n:COUNTRIES) RETURN id(n) AS id UNION ALL MATCH (s:STYLE) return id(s) as id' relationshipQuery='MATCH (c:COUNTRIES)<-[]-(ci:CITIES)<-[]-(br:BREWERIES)-[]->(be:BEERS)-[]->(s:STYLE) WITH c as Country, s as Style, count(*) as Freq\n                ORDER BY Freq DESC\n                WITH Country, collect(Style)[..10] as Top_10 UNWIND Top_10 as Country_Top_10 \n                RETURN id(Country) as source, id(Country_Top_10) as target' graphName='similar_countries' nodeCount=313 relationshipCount=1418 projectMillis=5270>]


In [21]:
# Step 2 - Run the Node Similarity algorithm

try:
    query_9c = """
        CALL gds.nodeSimilarity.stream('similar_countries') 
            YIELD node1, node2, similarity
            WITH gds.util.asNode(node1).name AS Country1, gds.util.asNode(node2).name AS Country2, similarity
            WHERE Country1 <> "nan" AND Country2 <> "nan"
            RETURN Country1, Country2, similarity
            ORDER BY similarity DESCENDING
            LIMIT 5
        """

    result_9c = execute_read(driver, query_9c)

    pprint(result_9c)
    
except Exception as e:
    pprint(e)

[<Record Country1='AU' Country2='NZ' similarity=1.0>,
 <Record Country1='CA' Country2='PH' similarity=1.0>,
 <Record Country1='NZ' Country2='AU' similarity=1.0>,
 <Record Country1='UA' Country2='LT' similarity=1.0>,
 <Record Country1='KY' Country2='ET' similarity=1.0>]


In [22]:
# Check top 10 beer styles for the 2 most similar countries

country = result_9c[0][0]
query_9_check1 = f"""
             MATCH (c:COUNTRIES{{name: "{country}"}})<-[in:IN]-(ci:CITIES)<-[ins:IN]-(br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[h:HAS_STYLE]->(st:STYLE)
             RETURN c.name as Country, st.name as Style, count(st.name) as Number_of_Beers
             ORDER BY Number_of_Beers DESC
             LIMIT 10
            """

result_9_check1 = execute_read(driver, query_9_check1)

pprint(result_9_check1)

[<Record Country='AU' Style='American IPA' Number_of_Beers=1893>,
 <Record Country='AU' Style='American Pale Ale (APA)' Number_of_Beers=1070>,
 <Record Country='AU' Style='American Imperial IPA' Number_of_Beers=867>,
 <Record Country='AU' Style='Belgian Saison' Number_of_Beers=770>,
 <Record Country='AU' Style='American Imperial Stout' Number_of_Beers=467>,
 <Record Country='AU' Style='American Porter' Number_of_Beers=452>,
 <Record Country='AU' Style='American Amber / Red Ale' Number_of_Beers=450>,
 <Record Country='AU' Style='American Wild Ale' Number_of_Beers=411>,
 <Record Country='AU' Style='American Stout' Number_of_Beers=383>,
 <Record Country='AU' Style='Fruit and Field Beer' Number_of_Beers=360>]


In [23]:
# Check top 10 beer styles for the 2 most similar countries


country = result_9c[0][1]
query_9_check2 = f"""
             MATCH (c:COUNTRIES{{name: "{country}"}})<-[in:IN]-(ci:CITIES)<-[ins:IN]-(br:BREWERIES)-[bre:BREWED]->(b:BEERS)-[h:HAS_STYLE]->(st:STYLE)
             RETURN c.name as Country, st.name as Style, count(st.name) as Number_of_Beers
             ORDER BY Number_of_Beers DESC
             LIMIT 10
            """

result_9_check2 = execute_read(driver, query_9_check2)

pprint(result_9_check2)

[<Record Country='NZ' Style='American IPA' Number_of_Beers=886>,
 <Record Country='NZ' Style='American Imperial IPA' Number_of_Beers=427>,
 <Record Country='NZ' Style='American Pale Ale (APA)' Number_of_Beers=383>,
 <Record Country='NZ' Style='Belgian Saison' Number_of_Beers=330>,
 <Record Country='NZ' Style='American Imperial Stout' Number_of_Beers=250>,
 <Record Country='NZ' Style='American Porter' Number_of_Beers=241>,
 <Record Country='NZ' Style='American Stout' Number_of_Beers=194>,
 <Record Country='NZ' Style='American Amber / Red Ale' Number_of_Beers=193>,
 <Record Country='NZ' Style='American Wild Ale' Number_of_Beers=168>,
 <Record Country='NZ' Style='Fruit and Field Beer' Number_of_Beers=161>]


In [24]:
# Checking the number of countries with a similarity equal to 1. We divided the number by 2 because each pair is repeated twice.

try:
    query_9c = """
        CALL gds.nodeSimilarity.stream('similar_countries') 
            YIELD node1, node2, similarity
            WITH gds.util.asNode(node1).name AS Country1, gds.util.asNode(node2).name AS Country2, similarity
            WHERE Country1 <> "nan" AND Country2 <> "nan" AND similarity = 1
            RETURN COUNT(*)/2 as Number_Similar_Countries
        """

    result_9c = execute_read(driver, query_9c)

    pprint(result_9c)
    
except Exception as e:
    pprint(e)

[<Record Number_Similar_Countries=106>]


<b>Answer 9.1:</b> There is 106 countries with the same top 10 styles. We chose randomly one pair and confirmed their top 10 styles.

#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?

<b>10.2.</b> Which user is the most influential when it comes to reviews made?

In [25]:
# Step 0 - Clear graph

try:
    query_10a = """
            CALL gds.graph.drop('user_influence') YIELD graphName;
        """

    result_10a = execute_read(driver, query_10a)

    pprint(result_10a)
    
except Exception as e:
    pprint(e)

[<Record graphName='user_influence'>]


In [26]:
# Step 1 - Create the user_influence graph

try:
    query_10b = """
        CALL gds.graph.project.cypher(
            'user_influence',
            "MATCH (n) where head(Labels(n))='USER' or head(Labels(n))='REVIEWS' RETURN id(n) AS id",
            "MATCH (u:USER)<-[]-(r:REVIEWS)
             RETURN id(r) AS source, id(u) AS target"
            )
            """

    result_10b = execute_read(driver, query_10b)

    pprint(result_10b)
    
except Exception as e:
    pprint(e)

[<Record nodeQuery="MATCH (n) where head(Labels(n))='USER' or head(Labels(n))='REVIEWS' RETURN id(n) AS id" relationshipQuery='MATCH (u:USER)<-[]-(r:REVIEWS)\n             RETURN id(r) AS source, id(u) AS target' graphName='user_influence' nodeCount=9238044 relationshipCount=9073109 projectMillis=30232>]


In [27]:
# Step 2 - Run the PageRank algorithm

try:
    query_10c = """
        CALL gds.pageRank.stream('user_influence') 
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS Username, ROUND(score,2) as Influence_Score
            ORDER BY Influence_Score DESC
            LIMIT 5
        """

    result_10c = execute_read(driver, query_10c)

    pprint(result_10c)
    
except Exception as e:
    pprint(e)

[<Record Username='Sammy' Influence_Score=1759.27>,
 <Record Username='kylehay2004' Influence_Score=1558.33>,
 <Record Username='acurtis' Influence_Score=1532.19>,
 <Record Username='StonedTrippin' Influence_Score=1512.17>,
 <Record Username='jaydoc' Influence_Score=1504.65>]


In [28]:
# Double check the users with most reviews

query_10_check = """
            MATCH (u:USER)<-[]-(r:REVIEWS)
            RETURN u.name as Username, COUNT(r) as Number_of_Reviews
            ORDER BY Number_of_Reviews DESC
            LIMIT 5
          """

result_10_check = execute_read(driver, query_10_check)

pprint(result_10_check)

[<Record Username='Sammy' Number_of_Reviews=13797>,
 <Record Username='kylehay2004' Number_of_Reviews=12221>,
 <Record Username='acurtis' Number_of_Reviews=12016>,
 <Record Username='StonedTrippin' Number_of_Reviews=11859>,
 <Record Username='jaydoc' Number_of_Reviews=11800>]


<b>Answer 10.2:</b> Sammy is the most influencial user when it comes to reviews made.

#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [29]:
# Getting the average reviews per beer

query_11extra1 = """
                 MATCH (r:REVIEWS)
                 WITH r.beer_id as Beer, COUNT(r) as Number_of_Reviews
                 RETURN ROUND(AVG(Number_of_Reviews), 2) as Average_Reviews
                """

result_11extra1 = execute_read(driver, query_11extra1)

pprint(result_11extra1)

[<Record Average_Reviews=29.31>]


In [30]:
# Getting the 99th percentile reviews per beer

query_11extra2 = """
                    MATCH (r:REVIEWS)
                    WITH r.beer_id as Beer, COUNT(r) as Number_of_Reviews
                    WITH percentileDisc(Number_of_Reviews, 0.99) as Minimum_Reviews, COUNT(DISTINCT Beer) as Total_Number_of_Beers
                    MATCH (b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
                    WITH b, COUNT(r) as Number_of_Reviews, Minimum_Reviews, Total_Number_of_Beers
                    WHERE Number_of_Reviews >= Minimum_Reviews 
                    RETURN Minimum_Reviews, Total_Number_of_Beers, COUNT(DISTINCT b.name) as Percentile_Number_of_Beers
                 """

result_11extra2 = execute_read(driver, query_11extra2)

pprint(result_11extra2)

[<Record Minimum_Reviews=502 Total_Number_of_Beers=309540 Percentile_Number_of_Beers=3043>]


In [31]:
query_11 = """
            MATCH (rev:REVIEWS)
            WITH rev.beer_id as beer, COUNT(rev) as Rev_Num
            WITH percentileDisc(Rev_Num, 0.99) as Minimum_Reviews
            MATCH (b:BEERS)-[rv:REVIEWED]->(r:REVIEWS)
            WITH b, COUNT(r) as Num_Rev, Minimum_Reviews
            WHERE Num_Rev >= Minimum_Reviews 
            MATCH (be:BEERS)-[rv:REVIEWED]->(re:REVIEWS)
            WHERE be.name IN b.name AND toFloat(re.score) <> 'nan'
            RETURN DISTINCT be.name as Beer, ROUND(AVG(toFloat(re.score)), 2) as Average_Score, COUNT(re) as Number_of_Reviews
            ORDER BY Average_Score DESC, Number_of_Reviews DESC
            LIMIT 3
            """

result_11 = execute_read(driver, query_11)

pprint(result_11)

[<Record Beer='Kentucky Brunch Brand Stout' Average_Score=4.84 Number_of_Reviews=702>,
 <Record Beer='Rare Bourbon County Brand Stout (2010)' Average_Score=4.8 Number_of_Reviews=1382>,
 <Record Beer='Good Morning' Average_Score=4.78 Number_of_Reviews=1039>]


<b>Answer:</b> We would recommend Kentucky Brunch Brand Stout, Rare Bourbon County Brand Stout (2010), and Good Morning.  
<b>Explanation:</b> We based our recommendation on the number of reviews (to avoid bias) and the score attribute as  it seems to represent the average of all the attributes and it appears in all the reviews (other attributes had nan's).  
In order to choose the minimum number of reviews, we calculated first the average number of reviews which was 29, and we got the beers with the highest score and a number of reviews higher than the average. But the average of reviews was relatively low.
As a second step we checked the median which was 3 reviews, then we tried with different percentiles and ended up choosing the 99th percentile giving the 1% of the beers with the highest number of reviews (3043 beers with a minimum number of reviews of 502).